In [292]:
import os
import sys
os.environ['TRANSFORMERS_CACHE'] = '/projectnb/cs505/mahirp/cache'
os.environ['PREFIX_PATH']="/projectnb/cs505/mahirp/python"
os.environ['TFHUB_CACHE_DIR']='/projectnb/cs505/mahirp/tfcache'
sys.path.append('/projectnb/cs505/mahirp/python/bin')
sys.path.append('/projectnb/cs505/mahirp/cache')
sys.path.append('/projectnb/cs505/mahirp/tfcache')

In [293]:
!pip install --ignore-installed --prefix=$PREFIX_PATH transformers sklearn



  Using cached transformers-4.5.1-py3-none-any.whl (2.1 MB)
Processing /usr4/cs640/mahirp/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e/sklearn-0.0-py2.py3-none-any.whl
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached tokenizers-0.10.2-cp37-cp37m-manylinux2010_x86_64.whl (3.3 MB)
  Using cached importlib_metadata-4.0.0-py3-none-any.whl (16 kB)
  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached regex-2021.4.4-cp37-cp37m-manylinux2014_x86_64.whl (720 kB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached tqdm-4.60.0-py2.py3-none-any.whl (75 kB)
  Using cached packaging-20.9-py2.py3-none-any.whl (40 kB)
  Using cached scikit_learn-0.24.1-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Using cached zipp-3.4.1-py3-none-any.whl (5.2 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached id

In [4]:
from transformers import pipeline, set_seed
import re
import pandas as pd
import tensorflow_hub as hub
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf


import numpy as np
loadFromFile=True

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/CS505/RelevanceDetector/')

In [6]:
prepend=datetime.now().strftime("%m-%d-%Y-%H-%M-%S-")
dataset = pd.read_excel('CS505-gunviolence/GVFC_AnnotatedHeadlines.xlsx', index_col=0)
if not loadFromFile:
    true=dataset[dataset['V3relevance']==1]
    false=dataset[dataset['V3relevance']==0]
    trainDataset=true.sample(frac=0.5)
    testDataset=true.drop(trainDataset.index)
    ftrainDataset=false.sample(frac=0.5)
    ftestDataset=false.drop(ftrainDataset.index)
    trainDataset=trainDataset.append(ftrainDataset).sample(frac=1)
    testDataset=testDataset.append(ftestDataset).sample(frac=1)
    indices=np.array(trainDataset.index)
    np.save(open('train_indices.npy','wb'),indices)
else:
    print('loaded from storage')
    indices=np.load(open('train_indices.npy','rb'))
    trainDataset=dataset.iloc[indices]
    testDataset=dataset.drop(indices)
print(len(trainDataset))
print(len(testDataset))

loaded from storage
650
650


In [7]:

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
generator = pipeline('text-generation', model='gpt2',device=0)
tokenizer=generator.tokenizer

INFO:absl:Using /projectnb/cs505/mahirp/tfcache/ to cache modules.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [8]:
prepend=datetime.now().strftime("%m-%d-%Y-%H-%M-%S-")
def generateDataset(dataset,isTest=False):
    df = pd.DataFrame(columns=['original','headline','generated','relevant'])
    c=0
    gen_seq=2 if not isTest else 1
    for original,headline,caption,relevant in zip(dataset['whole_text'],dataset['news_title'],dataset['generated_caption_of_lead_image'],dataset['V3relevance']):
        print("{}/{}".format(c,len(trainDataset)))
        c+=1
        text=generator(caption, max_length=min(100,len(original)), num_return_sequences=gen_seq)
        headLineText=generator(headline,max_length=min(100,len(original)),num_return_sequences=gen_seq)
        for t,h in zip(text,headLineText):
          df=df.append({'original':original,'headline':h['generated_text'],'generated': t['generated_text'],'relevant':relevant},ignore_index = True)
    return df

In [16]:
regexMap = { r"\n":" ",r"\\":" ",r"[\s]+": " "}
def cleanData(text):
    for regx in regexMap.keys():
        text = re.sub(regx, regexMap[regx], text)
    return text

In [11]:
def getEmbeddings(text):
  return embed([text]).numpy().astype(np.float)

In [ ]:
trainDF = generateDataset(trainDataset)
trainDF['generated']=list(map(cleanData,trainDF['generated']))
trainDF['headline']=list(map(cleanData,trainDF['headline']))
trainDF['generated']=list(map(getEmbeddings,trainDF['generated']))
print('done')
trainDF['headline']=list(map(getEmbeddings,trainDF['headline']))
print('done')
trainDF['original']=list(map(getEmbeddings,trainDF['original']))
trainDF.to_csv('CS505-gunviolence/'+prepend+'train_relevanceDatasetV3.csv',index=False)

In [ ]:
testDF = generateDataset(testDataset,True)
testDF['generated']=list(map(cleanData,testDF['generated']))
testDF['headline']=list(map(cleanData,testDF['headline']))
testDF['generated']=list(map(getEmbeddings,testDF['generated']))
print('done')
testDF['headline']=list(map(getEmbeddings,testDF['headline']))
print('done')
testDF['original']=list(map(getEmbeddings,testDF['original']))
testDF.to_csv('CS505-gunviolence/'+prepend+'test_relevanceDatasetV3.csv',index=False)

In [266]:
# df=pd.read_csv('CS505-gunviolence/'+prepend+'train_relevanceDatasetV3.csv')
df = pd.read_csv('CS505-gunviolence/04-19-2021-04-43-45-train_relevanceDatasetV3.csv')
labels=df.pop('relevant')

In [100]:
df.head()

,original,headline,generated,relevant
0,[[-5.22599556e-02 9.07361284e-02 -4.04457152e...,[[-0.0157384 0.05399382 -0.14654243 0.00188...,[[ 0.02107025 0.03470307 0.01478738 -0.00714...,0
1,[[-5.22599481e-02 9.07361135e-02 -4.04457226e...,[[-8.97298940e-03 3.86566184e-02 -1.10700771e...,[[ 2.21908875e-02 6.18911348e-02 1.50320400e...,0
2,[[-0.01018255 0.08759642 -0.05752711 -0.00277...,[[ 2.33617481e-02 -3.09682321e-02 -3.69818974e...,[[-0.02904347 0.03948796 0.01419673 0.06375...,0
3,[[-0.01018255 0.08759639 -0.05752707 -0.00277...,[[ 3.61642949e-02 1.24041177e-02 2.65971124e...,[[-4.78867516e-02 -6.19829409e-02 1.65775195e...,0
4,[[-4.80321720e-02 1.12838263e-03 -2.45341770e...,[[ 0.01234712 -0.07639864 0.01749216 -0.02077...,[[ 0.01378198 -0.06673057 0.04916419 -0.03802...,1


In [254]:
def getModel():
  inp=tf.keras.Input(shape=(3,512),name='f1')
  x=tf.keras.layers.Dense(512,activation='relu')(inp)
  x=tf.keras.layers.Flatten()(x)
  x=tf.keras.layers.Dense(1,activation='sigmoid')(x)
  return tf.keras.Model(inp,x)

In [255]:
model=getModel()

In [256]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "functional_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
f1 (InputLayer)              [(None, 3, 512)]          0         
_________________________________________________________________
dense_44 (Dense)             (None, 3, 512)            262656    
_________________________________________________________________
flatten_16 (Flatten)         (None, 1536)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 1537      
Total params: 264,193
Trainable params: 264,193
Non-trainable params: 0
_________________________________________________________________


In [257]:
t=df['original'][20]
regexMap = { r"\n":" ",r"[\s]+": ",",r"\[,":"["}
t=np.array(eval(cleanData(t)),dtype=np.float)
t.shape

(1, 512)

In [267]:
features=[]
for index,data in df.iterrows():
  c1=np.array(eval(cleanData(data['headline'])),dtype=np.float)
  c2=np.array(eval(cleanData(data['generated'])),dtype=np.float)
  c3=np.array(eval(cleanData(data['original'])),dtype=np.float)
#   features.append(np.vstack((c1,c2)))
  features.append(np.vstack((c1,c2,c3)))

In [274]:
features=np.array(features,dtype=np.float)


In [275]:
features.shape

(1300, 3, 512)

In [276]:
dataset = tf.data.Dataset.from_tensor_slices(({'f1':features}, labels.values)).batch(1)


In [277]:
l=len(dataset)
dataset=dataset.shuffle(int(l*0.80))
train_set=dataset.take(int(l*0.75))
test_set=dataset.skip(int(l*0.75))
val_set=test_set.take(int(l*0.75*0.30))
test_set=test_set.skip(int(l*0.75*0.30))
len(test_set)

33

In [278]:
history=model.fit(train_set,validation_data=val_set,epochs=20)

Epoch 1/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5121 - accuracy: 0.9631 - val_loss: 0.5210 - val_accuracy: 0.9589
Epoch 2/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5132 - accuracy: 0.9600 - val_loss: 0.5205 - val_accuracy: 0.9692
Epoch 3/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5142 - accuracy: 0.9631 - val_loss: 0.5082 - val_accuracy: 0.9692
Epoch 4/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5034 - accuracy: 0.9692 - val_loss: 0.5259 - val_accuracy: 0.9521
Epoch 5/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5086 - accuracy: 0.9662 - val_loss: 0.5133 - val_accuracy: 0.9555
Epoch 6/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5098 - accuracy: 0.9651 - val_loss: 0.5135 - val_accuracy: 0.9521
Epoch 7/20
975/975 [==============================] - 1s 1ms/step - loss: 0.5148 - accuracy: 0.9672 - val_loss: 0.4902 - val_accuracy: 0.9829
Epoch 

In [279]:
x=model.evaluate(test_set)

33/33 [==============================] - 0s 788us/step - loss: 0.4702 - accuracy: 0.9697


In [62]:
model.save(prepend+'3relevanceClassifier.h5')

In [280]:
results={}
sampleCount=0
testDF = pd.read_csv('CS505-gunviolence/04-19-2021-04-43-45-test_relevanceDatasetV3.csv')
features=[]
labels=testDF.pop('relevant')
for index,data in testDF.iterrows():
  c1=np.array(eval(cleanData(data['headline'])),dtype=np.float)
  c2=np.array(eval(cleanData(data['generated'])),dtype=np.float)
#   features.append(np.vstack((c1,c2)))  
  c3=np.array(eval(cleanData(data['original'])),dtype=np.float)
  features.append(np.vstack((c1,c2,c3)))
features=np.array(features,dtype=np.float)

In [281]:
# sample=testDataset.sample(n=10)
# set_seed(1029102)
# output=[]
# i=1
# test_input=[]
# test_label=[]
# for original,headline,caption,relevant in zip(sample['whole_text'],sample['news_title'],sample['generated_caption_of_lead_image'],sample['V3relevance']):
#     print(i)
#     i+=1
#     text=generator(caption, max_length=min(100,len(original)))
#     headlineText=generator(headline, max_length=min(64,len(original)))
#     originalEmbedding=embed([original]).numpy()
#     textEmbedding=embed([text[0]['generated_text']]).numpy()
#     headlineEmbedding=embed([headlineText[0]['generated_text']]).numpy()
#     test_input.append(np.vstack((headlineEmbedding,textEmbedding,originalEmbedding)))
#     test_label.append(relevant)



In [282]:
# test_input=np.array(test_input,dtype=np.float)
# test_label=np.array(test_label)
print(features.shape)
test_dataset = tf.data.Dataset.from_tensor_slices(({'f1':features}, labels)).batch(1)
x=model.evaluate(test_dataset)

(650, 3, 512)
650/650 [==============================] - 0s 758us/step - loss: 0.6752 - accuracy: 0.6308


In [249]:
results[sampleCount]=x
sampleCount+=1

In [294]:
trainDF = pd.read_csv('CS505-gunviolence/04-19-2021-04-43-45-train_relevanceDatasetV3.csv')
testDF = pd.read_csv('CS505-gunviolence/04-19-2021-04-43-45-test_relevanceDatasetV3.csv')
from sklearn.
trainDF.head()

ModuleNotFoundError: No module named 'sklearn.metrics.pairwise.cosine_similarity'; 'sklearn.metrics.pairwise' is not a package

In [ ]:
features=[]
for index,data in trainDF.iterrows():
  c1=np.array(eval(cleanData(data['headline'])),dtype=np.float)
  c2=np.array(eval(cleanData(data['generated'])),dtype=np.float)
  c3=np.array(eval(cleanData(data['original'])),dtype=np.float)
  print(cosine_similarity(c1,c2),data[''])
#   features.append(np.vstack((c1,c2)))
#   features.append(np.vstack((c1,c2,c3)))